In [ ]:
from curses.ascii import ctrl
from ucl.common import byte_print, decode_version, decode_sn, getVoltage, pretty_print_obj, lib_version
from ucl.lowState import lowState
from ucl.lowCmd import lowCmd
from ucl.unitreeConnection import unitreeConnection, LOW_WIFI_DEFAULTS, LOW_WIRED_DEFAULTS
from ucl.enums import GaitType, SpeedLevel, MotorModeLow
from ucl.complex import motorCmd, motorCmdArray
import time
import sys
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from datetime import datetime
import glob

import threading

## Import Torch for Infer
import torch
import sys
import os

## Import Legged Loco Policy Inference Utils for utilizing the trained policy
from legged_loco_policy.ultra_simple_policy_base import *



class Robot :
    
    ## ================================ Policy Related Functions ================================ ##
    def __init_policy__(self):
        # Policy 및 추론기 Load
        self.actor = load_go1_base_policy("./legged_loco_policy/nodepth_model_6000.pt", device="cpu")

        # 테스트 observation 더미 초기화
        self.observation = {
        'base_ang_vel': [0.1, -0.05, 0.02],
        'base_rpy': [0.05, 0.1, 0.0],
        'velocity_commands': [0.5, 0.0, 0.0],
        'joint_pos': [0.0] * 12,
        'joint_vel': [0.1] * 12,
        'actions': [0.0] * 12,
        }        
        
        # 액션 예측
        action = self.actor(self.observation)
        
        print(f"예측된 (테스트) 액션: {action}")

        print("✓ actor load 완료!")
        
        self.motion = np.zeros(12)
        # 데이터 구조화 - 일단 빈값 넣기
        self.velocity_commands = [0,0,0]

    def infer_policy(self):
        ## Observation 파싱 및 Infer 부분
        # self.retrive_data() # 먼저 self.lstate 에 최신패킷 넣고 <-- Motor 가 하니 하면 안됨!

        # 데이터 구조화 - 일단 빈값 넣기
        # self.velocity_commands = [0,0,0]

        joint_pos_rel = self.q_list - self.default_joint_pos # last actions 기준 상대 위치 계산
        self.last_actions =  np.clip(np.copy(self.actions), -10,10) # Clip last actions with some margin (-20,20)
        
        #  observation 값 할당하기
        self.observation = {
            'base_ang_vel': list(self.gyroscope), # base angular velocity
            'base_rpy': list(self.rpy), # Roll, Pitch, Yaw
            'velocity_commands': list(self.velocity_commands), # 외부에서 주는 명령 속도 (제어값임!!!)
            'joint_pos': list(joint_pos_rel), # Joint 값은 상대위치값을 사용함!
            'joint_vel': list(self.dq_list), # 관절 속도
            'actions': list(self.last_actions), # 마지막 액션값
            }

        ## Finally... Inference
        self.actions = np.array(predict_action(self.observation))# Action 위치값 날것 출력
        
        action_rate_limit = np.deg2rad(200) # Prevent from too much violent movement
        self.actions =  np.clip(self.actions, -action_rate_limit,action_rate_limit) # Clip curr actions with some margin 

    def postprocess_action_2_motion(self) :
        # Action 값을 Motion Command 로 변환함
        motion = self.actions * 0.25 # motion Scaling(원래있음)
        motion = motion + self.default_joint_pos # 상대값인 Action 에 관절 위치값 더하기
        self.motion = motion * 1
        return motion
    ## ================================ Policy Related Functions ================================ ##

    def __init__(self):
        ## Import Policy Model
        self.__init_policy__()
                
        
        ## Initialize connection ##
        print(f'Running lib version: {lib_version()}')
        self.conn = unitreeConnection(LOW_WIFI_DEFAULTS)
        self.conn.startRecv()
        ## == Initialize connection == ##

        ## instantiate lowlevel command and state objects ##
        self.lcmd = lowCmd()
        self.lstate = lowState()
        self.mCmdArr = motorCmdArray()


        # 로봇 제어에 사용되는 딕셔너리, lowcmd에서 이미 지정된 순서서
        self.d = {'FR_0':0, 'FR_1':1, 'FR_2':2,
                'FL_0':3, 'FL_1':4, 'FL_2':5,
                'RR_0':6, 'RR_1':7, 'RR_2':8,
                'RL_0':9, 'RL_1':10, 'RL_2':11 }

        # !!! 잘못된값 !!! IsaacLab 에서 사용되는 Joint 순서, 제어시 사용함
        # self.joint = [
        #     'FL_0', 'FL_1', 'FL_2', # FL_hip, FL_thigh, FL_calf
        #     'FR_0', 'FR_1', 'FR_2', # FR_hip, FR_thigh, FR_calf
        #     'RL_0', 'RL_1', 'RL_2', # RL_hip, RL_thigh, RL_calf
        #     'RR_0', 'RR_1', 'RR_2'  # RR_hip, RR_thigh, RR_calf
        #     ]        
        # IsaacLab 에서 사용되는 Joint 순서, 제어시 사용함
        self.joint = [
            'FL_0', 'FR_0', 'RL_0', 'RR_0',
            'FL_1', 'FR_1', 'RL_1', 'RR_1',
            'FL_2', 'FR_2', 'RL_2', 'RR_2'            
            ]
        
        self.default_joint_pos = np.array([
             0.1, -0.1,  0.1, -0.1,
             0.8,  0.8,  1.0,  1.0,
            -1.5, -1.5, -1.5, -1.5
        ])
        

        ## get initial state & print Log ##
        # Send empty command to tell the dog the receive port and initialize the connection
        cmd_bytes = self.lcmd.buildCmd(debug=False)
        self.conn.send(cmd_bytes)


        ## Robot Control Parameters
        # control HZ : 50Hz
        self.ctrldt = 1/50 # Ctrl loop HZ : 50Hz
        # PD Parameters
        self.kp = 20
        self.kd = 4
        # Motor Torque Limit
        self.tau = [0]*12

        ## Define Default Robot Status values
        # robot status
        self.q_list = np.zeros(12) #joint pos
        self.dq_list = np.zeros(12) #joint vel
        self.rpy = np.zeros(3) #base rpy
        self.gyroscope = np.zeros(3) #base ang vel
        self.accelerometer = np.zeros(3) #base lin acc
        self.quaternion = np.zeros(4) #base quat

        # Calculat'd
        self.base_lin_vel = np.zeros(3) # base lin vel
        self.projected_gravity = np.zeros(3) # projected gravity
        self.filtered_accel = np.zeros(3)

        # Policy rel.
        self.actions = np.zeros(12) # actions(Current Target)
        self.last_actions = np.zeros(12) # last actions(Previous Target)

        # Control Target
        self.velocity_commands = np.zeros(3) # velocity commands

        # Timestamp
        self.timestamp = time.time() # Initialize timestamp
        
        # Flag if robot is Stand up State
        self.is_standup = False

        # Flag if permitted process policy
        self.do_inference = True
        # Motion For Ctrl Robot
        # Already declared at __init_policy__
        # self.motion = np.zeros(12)
        self.velocity_commands = [0,0,0]

    ## define a function to print log data
    # 나중에 안쓰는 메서드일듯
    def printLog(self):
        print(f'Cycles:\t\t\t{self.lstate.bms.cycle}')
        data = self.conn.getData()
        packet = data[-1] #Get Last Data
        print('+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=')
        self.lstate.parseData(packet)

        print(f'SN [{byte_print(self.lstate.SN)}]:\t{decode_sn(self.lstate.SN)}')
        print(f'Ver [{byte_print(self.lstate.version)}]:\t{decode_version(self.lstate.version)}')
        print(f'SOC:\t\t\t{self.lstate.bms.SOC} %')
        print(f'Overall Voltage:\t{getVoltage(self.lstate.bms.cell_vol)} mv') #something is still wrong here ?!
        print(f'Current:\t\t{self.lstate.bms.current} mA')
        print(f'Temps BQ:\t\t{self.lstate.bms.BQ_NTC[0]} °C, {self.lstate.bms.BQ_NTC[1]}°C')
        print(f'Temps MCU:\t\t{self.lstate.bms.MCU_NTC[0]} °C, {self.lstate.bms.MCU_NTC[1]}°C')
        print(f'FootForce:\t\t{self.lstate.footForce}')
        print(f'FootForceEst:\t\t{self.lstate.footForceEst}')
        print(f'IMU Temp:\t\t{self.lstate.imu.temperature}')
        print(f'MotorState FR_0 MODE:\t\t{self.lstate.motorState[self.d["FR_0"]].mode}')
        print('+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=')

    def retrive_data(self):
        """
        conn에서 최신 패킷(가장 마지막)을 받아 lstate에 저장
        이후 값 파싱해서 self.* 에 저장
        """
        data = self.conn.getData()
        self.lstate.parseData(data[-1])
        self.q_list = np.array([self.lstate.motorState[self.d[joint]].q for joint in self.joint]) # 현재 관절위치 (절대위치!!!!)
        self.q_list_rel = np.array(self.default_joint_pos) - self.q_list # 현재 관절위치 상대위치

        self.dq_list = np.array([self.lstate.motorState[self.d[joint]].dq for joint in self.joint])

        # process IMU data
        self.rpy = np.array(self.lstate.imu.rpy)
        self.quaternion = np.array(self.lstate.imu.quaternion)
        self.gyroscope = np.array(self.lstate.imu.gyroscope)
        self.accelerometer = np.array(self.lstate.imu.accelerometer)

        # Calculate Extra Values 
        self.projected_gravity = self.estim_projected_gravity()
        self.estimate_lin_vel()

    def estim_projected_gravity(self):
        """
        projected_gravity 계산 - Quaternion 우선 사용!
        """
        # Quaternion 사용
        w, x, y, z = self.quaternion
        return np.array([
            2 * (x * z - w * y),
            2 * (w * x + y * z),
            w**2 - x**2 - y**2 + z**2
        ])
    
        
    def estimate_lin_vel(self):
        """
        가속도 적분으로 base_lin_vel 추정 (비추천)
        
        Args:
            accelerometer: (3,) 현재 가속도
            quaternion: (w, x, y, z)
            dt: 시간 간격 (예: 0.02초)
            prev_velocity: 이전 속도
            prev_filtered_accel: 이전 필터링된 가속도
            alpha: 필터 계수
            decay: 드리프트 감쇠
        
        Returns:
            velocity: (3,) 추정 속도
            filtered_accel: (3,) 필터링된 가속도
        """

        ## Set Default Values
        self.alpha = 0.7
        self.decay = 0.7
        
        # Set Previous Values
        prev_velocity = self.base_lin_vel * 1 # preveny var.linking
        prev_filtered_accel = self.filtered_accel * 1 # preveny var.linking

        # 1. projected_gravity
        pg = self.estim_projected_gravity()


        # 2. 중력 제거
        linear_accel = np.array(self.accelerometer) - pg * 9.81
        
        # 3. 필터링
        filtered_accel = self.alpha * np.array(prev_filtered_accel) + (1 - self.alpha) * linear_accel
        
        # 4. 적분
        velocity = prev_velocity + filtered_accel * self.ctrldt 
        
        # 5. 드리프트 보정
        velocity *= self.decay
        velocity[2] = 0 # Z축속도 0 으로 고정
        
        # Write output to class variables
        self.base_lin_vel = velocity 
        # self.base_lin_vel = np.array([0,0,0])
        self.filtered_accel = filtered_accel


    """
    아래 메서드들 이미 retrive_data 에서 처리됨, 없애도될듯?
    """
    def get_joint_pos(self):
        """
        self.lstate에서 최신 패킷을 받아 12개 관절의 각도(q, radian) 리스트를 반환
        --> **joint_pos** @ IsaacLab

        Returns:
            q_list (np.ndarray): 12개 관절의 각도 리스트 (radian, joint 순서는 global 'joint' 리스트와 동일)
                        오류시 None 반환
        """
        self.q_list = np.array([self.lstate.motorState[self.d[joint]].q for joint in self.joint])
        return self.q_list
    
    def get_joint_vel(self):
        """
        self.lstate에서 최신 패킷을 받아 12개 관절의 속도(dq, radian/s) 리스트를 반환
        --> **joint_vel** @ IsaacLab
        """
        self.dq_list = np.array([self.lstate.motorState[self.d[joint]].dq for joint in self.joint])
        return self.dq_list

    def get_rpy(self):
        """
        self.lstate에서 최신 패킷을 받아 로봇의 imu RPY 각도(roll, pitch, yaw, radian) 리스트를 반환
        --> **base_rpy** @ IsaacLab
        """
        self.rpy = np.array(self.lstate.imu.rpy)
        return self.rpy
    
    def get_gyroscope(self):
        """
        self.lstate에서 최신 패킷을 받아 로봇의 imu 각속도(roll, pitch, yaw, radian/s) 리스트를 반환
        --> **base_gyroscope** @ IsaacLab
        """
        self.gyroscope = np.array(self.lstate.imu.gyroscope)
        return self.gyroscope

    def move_motor(self,target_pos):
        """
        관절들들을 `target_pos` 로 이동
        """
        joint = self.joint
        for i in range(len(joint)):
            self.mCmdArr.setMotorCmd(joint[i],  motorCmd(mode=MotorModeLow.Servo, q=target_pos[i], dq = 0, Kp = self.kp, Kd = self.kd, tau = self.tau[i]))
        
        self.lcmd.motorCmd = self.mCmdArr
        cmd_bytes = self.lcmd.buildCmd(debug=False)
        self.conn.send(cmd_bytes)
    
    def do_motion(self,init_pos,target_pos,duration,debug=False):
        """
        특정 관절을 `init_pos` 에서 `target_pos` 로 선형 보간하여 이동

        Args:
            init_pos (list): 초기 관절 위치
            target_pos (list): 타겟 관절 위치
            duration (float): 이동 시간 (초)
        """
        hz = 1/self.ctrldt
        motion_step = int(duration*hz)
        print(f"do motion while : {motion_step/hz} seconds")
        for i in range(motion_step):
            self.retrive_data()
            qDes = self.jointLinearInterpolation(init_pos,target_pos,i/motion_step)
            qDes = list(qDes)
            if debug:
                # print(f"target : {self.rad2deg(qDes,2)}")
                print(f"target : {qDes}")
                print(f"Step : {i}/{motion_step}")
            else:
                # print(f"target : {self.rad2deg(qDes,2)}")
                print(f"target : {qDes}")
                print(f"Step : {i}/{motion_step}")
                self.move_motor(qDes)

            time.sleep(self.ctrldt)

    ## 별도의 쓰레드에서 loop_func(루프문) 함수 실행, Infrerence Loop 용
    def follow_target_pos(self,Threading = False):
        """
        별도의 쓰레드에서 loop_func(루프문) 함수 실행
        loop_func: 루프를 돌릴 함수. self를 인자로 받거나, 필요한 경우 *args, **kwargs로 인자 전달
        """

        def robot_control_loop():
            while True : 
                # check timestamp
                
                
                if not self.is_standup :
                    # 처음에 일어서기 동작 수행
                    print("Standing Up...")
                    ## Get Current Position
                    self.retrive_data()
                    initpos = self.q_list
                    self.motion = np.array(self.default_joint_pos) * 1 
                    
                    time.sleep(1/50)
                    self.do_motion(initpos,self.motion,1,debug=False)
                    print("wait 3sec... do orgnization!!")
                    self.do_motion(self.motion,self.motion,3,debug=False)
                    
                    self.is_standup = True
                    print("Stand Up Complete.")
                
                else : # Do Inference Loop
                    self.timestamp = time.time()
                    self.retrive_data() # Refresh Data
                    ## 1. Do Inference Policy HERE!
                    # >>> infer(observations, actions) 
                    # robot.retrive_data() # 먼저 self.lstate 에 최신패킷 넣고
                    # self.do_inference = True
                    if self.do_inference :
                        robot.infer_policy()
                        self.motion = list(robot.postprocess_action_2_motion())
                        print("currpose : ",self.q_list)
                        print("motion : ",self.motion)
                        ## 2. Move Motor to Target Action
                    
                    self.move_motor(self.motion)
                    # self.move_motor(self,self.actions)
                    
                    ## 
                    # Do something HERE!
                    ## 
                    ## DEBUG
                    ## 
                    loop_delay = self.ctrldt - (time.time() - self.timestamp) # 루프 지연시간이 보상된 sleeptime
                    print(f"Loop Time : {self.ctrldt - loop_delay:.4f} sec, Sleep Time : {loop_delay:.4f} sec")
                    if loop_delay >= 0 :
                        time.sleep(loop_delay)
                    else : 
                        print("Caution; looptime over : ", round(loop_delay,2))

        if Threading is True :
            self.control_thread = threading.Thread(target=robot_control_loop, daemon=True)
            self.control_thread.start()
        else :
            robot_control_loop()



    ## Utils
    def deg2rad(self,deg,digit=0):
        """
        numpy array 또는 list 형식의 각도를 radian으로 변환
        """
        deg = np.array(deg)
        if digit != 0:
            result = np.array([round(math.radians(d),digit) for d in deg])
        else:
            result = np.array([math.radians(d) for d in deg])
        return result
    
    def rad2deg(self,rad,digit=0):
        """
        numpy array 또는 list 형식의 각도를 도로 변환
        """
        rad = np.array(rad)
        if digit != 0:
            result = np.array([round(math.degrees(r),digit) for r in rad])
        else:
            result = np.array([math.degrees(r) for r in rad])
        return result


    ## Linear interpolation between two joint positions
    # Input : initPos(rate=0) ~~~~~~ targetPos(rate=1)
    # Output : interpolated_position =(p)
    def jointLinearInterpolation(self,initPos, targetPos, rate):
        rate = np.fmin(np.fmax(rate, 0.0), 1.0)
        p = initPos*(1-rate) + targetPos*rate
        return p

# 제어 인스턴스 생성
robot = Robot()
time.sleep(0.02)

robot.retrive_data() # 먼저 self.lstate 에 최신패킷 넣고

# 그다음 값 파싱해서 쓸것
print("default_joint_pos : ",robot.rad2deg(robot.default_joint_pos,2))
print("joint_pos : ",robot.rad2deg(robot.get_joint_pos(),2))
print("joint_vel : ",robot.rad2deg(robot.get_joint_vel(),2))
print("rpy : ",robot.rad2deg(robot.get_rpy(),2))
print("gyroscope : ",robot.rad2deg(robot.get_rpy(),2))

체크포인트 로딩: ./legged_loco_policy/nodepth_model_6000.pt
actor_input_size: 45
critic_input_size: 235
모델 로딩 완료!
예측된 (테스트) 액션: [-0.26546094  0.84305525 -0.63493645 -0.3726606   0.3098177  -0.6978923
 -0.9438715  -0.46007326  1.0673234   3.6245792   1.077427    0.11313675]
✓ actor load 완료!
Running lib version: 0.2
default_joint_pos :  [  5.73  -5.73   5.73  -5.73  45.84  45.84  57.3   57.3  -85.94 -85.94
 -85.94 -85.94]
joint_pos :  [  20.54  -18.32   18.78  -18.03   66.68   67.65   67.82   67.58 -159.42
 -158.25 -158.47 -164.88]
joint_vel :  [-0.25  0.08  0.62 -0.51  0.04  0.2  -0.84 -0.08 -0.11  0.04 -0.12 -0.  ]
rpy :  [ 0.25 -0.02  2.35]
gyroscope :  [ 0.25 -0.02  2.35]


체크포인트 로딩: ./legged_loco_policy/nodepth_model_6000.pt
actor_input_size: 45
critic_input_size: 235
모델 로딩 완료!
예측된 액션: [-0.26546094  0.84305525 -0.63493645 -0.3726606   0.3098177  -0.6978923
 -0.9438715  -0.46007326  1.0673234   3.6245792   1.077427    0.11313675]


In [45]:
robot.is_standup = False
robot.do_inference = True

robot.velocity_commands = np.array([1.0,0,0])
robot.follow_target_pos()

Standing Up...
do motion while : 1.0 seconds
target : [0.3584859371185303, -0.319851815700531, 0.32784509658813477, -0.31464409828186035, 1.1638076305389404, 1.180702567100525, 1.1837908029556274, 1.1794308423995972, -2.7823433876037598, -2.7619969844818115, -2.765791893005371, -2.8777379989624023]
Step : 0/50
target : [0.3533162183761597, -0.3154547793865204, 0.32328819465637204, -0.31035121631622314, 1.1565314779281617, 1.1730885157585145, 1.1801149868965148, 1.1758422255516052, -2.756696519851684, -2.736757044792175, -2.7404760551452636, -2.850183238983154]
Step : 1/50
target : [0.3481464996337891, -0.3110577430725098, 0.31873129272460937, -0.30605833435058594, 1.1492553253173827, 1.1654744644165038, 1.1764391708374022, 1.1722536087036133, -2.7310496520996095, -2.711517105102539, -2.715160217285156, -2.8226284790039062]
Step : 2/50
target : [0.3429767808914184, -0.30666070675849916, 0.31417439079284665, -0.30176545238494873, 1.141979172706604, 1.1578604130744934, 1.1727633547782899,

IndexError: list index out of range

In [ ]:
## Observation  Monitoring
import time
def r2d(r) :
    return robot.rad2deg(r,2)
while True : 
    robot.retrive_data()
    joint_pos_rel = robot.q_list - robot.default_joint_pos # last actions 기준 상대 위치 계산)
    print("===============================================================================================")
    print("linvel",robot.base_lin_vel) # base linear velocity; 추정값 !!!
    print("gyro",(robot.gyroscope)) # base angular velocity
    print("grav",robot.projected_gravity) # projected gravity; 추정값 !!!
    print("rpy",(robot.rpy)) # Roll, Pitch, Yaw
    print("qrel",(joint_pos_rel)) # joint rel pose
    print("cmdvel",robot.velocity_commands) # 외부에서 주는 명령 속도 (제어값임!!!)
    print("dq",(robot.dq_list)) # 관절 속도
    print("action",(robot.last_actions)) # 마지막 액션값
    print("===============================================================================================")
    time.sleep(0.1)

# dummy_obs = [ 0.8249, -0.0841,  0.0609, lin vel  
    #          0.3209,  0.2600,  0.1070,  ang vel
    #          0.0558, -0.0461,-0.9974,  proj grav
    #          0.0462,  0.0558,  2.6461, rpy
    #          0.7924,  0.0000,  0.0595, cmd_vel
    #         -0.3614, 0.3551, -0.5186,  0.4916,  0.0299, -0.4008, -0.9144, -0.3664, -0.1314, 0.2164,  0.4206,  0.1846, # curr_joint_pos
    #         -1.5743,  0.2659,  0.0053, -0.1578, -1.1460, 1.1299,  3.0969,  1.6914, -1.2098, 1.2192, -2.4923,  0.7213, # cur_joiint_vel
    #         -1.5323, 1.4793, -2.1506,  1.6954, -0.6748, -1.1294, -3.2514, -1.4343, -0.9111, 2.1069,  2.2329,  1.4590, # prev_action 

In [25]:
while True :
    robot.retrive_data() # 먼저 self.lstate 에 최신패킷 넣고
    robot.infer_policy()
    motion = robot.postprocess_action_2_motion()
    print("Inferered actions : ",robot.rad2deg(robot.actions,2))
    print("Current joint pos : ",robot.rad2deg(robot.get_joint_pos(),2))
    print("inferred Motions : ",robot.rad2deg(motion,2))
    print("action diff rel to current pos : ",robot.rad2deg(motion - robot.get_joint_pos(),2))
    time.sleep(0.1)

Inferered actions :  [ 79.24 -18.64  61.26 -21.33  28.57  57.14 -80.    18.8  -80.   -13.43
 -72.33 -54.48]
Current joint pos :  [-23.95  36.94 -24.81  37.33  26.59  12.59  31.16  21.47 -52.36 -56.01
 -90.12 -58.37]
inferred Motions :  [  25.54  -10.39   21.04  -11.06   52.98   60.12   37.3    62.   -105.94
  -89.3  -104.03  -99.56]
action diff rel to current pos :  [ 49.49 -47.33  45.85 -48.39  26.39  47.53   6.14  40.53 -53.58 -33.3
 -13.9  -41.19]
Inferered actions :  [ 80.    48.15  80.   -24.31  80.    80.   -80.    71.95 -80.    17.56
 -80.   -80.  ]
Current joint pos :  [-23.95  36.94 -24.81  37.32  26.59  12.59  31.16  21.47 -52.36 -56.01
 -90.12 -58.37]
inferred Motions :  [  25.73    6.31   25.73  -11.81   65.84   65.84   37.3    75.28 -105.94
  -81.55 -105.94 -105.94]
action diff rel to current pos :  [ 49.68 -30.64  50.54 -49.13  39.25  53.24   6.13  53.81 -53.58 -25.55
 -15.82 -47.57]
Inferered actions :  [ 54.14  68.46  80.     7.53  80.    19.33 -80.    40.55 -80.    80.

KeyboardInterrupt: 

In [305]:
robot.is_standup = False
robot.follow_target_pos()

Standing Up...
do motion while : 1.0 seconds
target : [0.8609718084335327, 0.604460597038269, -2.297055244445801, -0.8326926231384277, 0.726963460445404, -2.2906768321990967, 0.6661661863327026, 0.9653687477111816, -2.544240951538086, -0.6323158740997314, 1.003336787223816, -2.6721737384796143]
Step : 0/50
target : [0.845752372264862, 0.6083713850975037, -2.2811141395568844, -0.8180387706756592, 0.7284241912364959, -2.2748632955551145, 0.6548428626060486, 0.966061372756958, -2.523356132507324, -0.6216695566177368, 1.0032700514793396, -2.648730263710022]
Step : 1/50
target : [0.8305329360961914, 0.6122821731567383, -2.265173034667969, -0.8033849182128906, 0.7298849220275879, -2.2590497589111327, 0.6435195388793945, 0.9667539978027344, -2.5024713134765624, -0.6110232391357422, 1.0032033157348632, -2.6252867889404294]
Step : 2/50
target : [0.8153134999275207, 0.6161929612159729, -2.2492319297790524, -0.7887310657501221, 0.7313456528186798, -2.2432362222671505, 0.6321962151527405, 0.967446

KeyboardInterrupt: 

In [ ]:
from legged_loco_policy.ultra_simple_policy_base import *
# 정책 로드
predict_action = load_go1_base_policy("./legged_loco_policy/nodepth_model_6000.pt", device="cpu")

# 테스트 observation
observation = {
'base_ang_vel': [0.1, -0.05, 0.02],
'base_rpy': [0.05, 0.1, 0.0],
'velocity_commands': [0.5, 0.0, 0.0],
'joint_pos': [0.0] * 12,
'joint_vel': [0.1] * 12,
'actions': [0.0] * 12,
}

# 액션 예측
action = predict_action(observation)
print(f"예측된 액션: {action}")

In [ ]:
## For State Logging 
# Default Power-on Joint Position (rad): [0.36284593,  1.17937028, -2.79138637, -0.34794939,  1.17319369, -2.74948239,  0.37132359,  1.18784809, -2.75150084, -0.27728164, 1.19438803, -2.88072538]
# Default Joint Position Position (deg): [  20.8   67.6 -159.9  -19.9   67.2 -157.5   21.3   68.1 -157.6  -15.9  68.4 -165.1]
# Calf Joint Range : -160(Folded) ~ -50(Unfolded) = 110 Deg
# Ankle Joint Range : -90(Folded) ~ 0(Unfolded) = 90 Deg 
robot.retrive_data()
list(robot.get_joint_pos())

[0.3468593955039978,
 1.1701054573059082,
 -2.785048246383667,
 -0.29411593079566956,
 1.2166723012924194,
 -2.7780239582061768,
 0.24760961532592773,
 1.206438422203064,
 -2.7810919284820557,
 -0.30931520462036133,
 1.1904518604278564,
 -2.8800792694091797]

In [ ]:
## Standup Motion 개발 흔적임
robot.retrive_data() # 먼저 self.lstate 에 최신패킷 넣고

initpos = robot.get_joint_pos()
targetpos = np.copy(initpos)

# targetpos[0:3] = [0.1,0.8,-1.5]  # FL_hip, FL_thigh, FL_calf
targetpos = robot.default_joint_pos
# print("initpos : ",robot.rad2deg(initpos,2))
# print("targetpos : ",robot.rad2deg(targetpos,2))
print("initpos : ",initpos)
print("targetpos : ",targetpos)
print("motion Diff (deg) : ",robot.rad2deg(targetpos - initpos,2))

initpos :  [ 0.34691992  1.17010546 -2.78508878 -0.29411593  1.2166723  -2.77798343
  0.24760962  1.20643842 -2.78105164 -0.30937576  1.19045186 -2.88007927]
targetpos :  [ 0.1  0.8 -1.5 -0.1  0.8 -1.5  0.1  1.  -1.5 -0.1  1.  -1.5]
motion Diff (deg) :  [-14.15 -21.21  73.63  11.12 -23.87  73.22  -8.46 -11.83  73.4   12.
 -10.91  79.07]


In [143]:
## 조인트값 모니터링용
while True :
    robot.retrive_data() # 먼저 self.lstate 에 최신패킷 넣고
    print(robot.rad2deg(robot.get_joint_pos(),1))
    time.sleep(0.2)

[  23.6   65.9 -157.5  -18.1   68.4 -157.9   23.6   75.2 -158.1  -21.7
   80.9 -165.2]
[  23.6   65.9 -157.5  -18.1   68.4 -157.9   23.6   75.2 -158.1  -21.7
   80.9 -165.2]
[  23.6   65.9 -157.5  -18.1   68.4 -157.9   23.6   75.2 -158.1  -21.7
   80.9 -165.2]
[  23.6   65.9 -157.5  -18.1   68.4 -157.9   23.6   75.2 -158.1  -21.7
   80.9 -165.2]
[  23.6   65.9 -157.5  -18.1   68.4 -157.9   23.6   75.2 -158.1  -21.7
   80.9 -165.2]
[  23.6   65.9 -157.5  -18.1   68.4 -157.9   23.6   75.2 -158.1  -21.7
   80.9 -165.2]
[  23.6   65.9 -157.5  -18.1   68.4 -157.9   23.6   75.2 -158.1  -21.7
   80.9 -165.2]
[  23.6   65.9 -157.5  -18.1   68.4 -157.9   23.6   75.2 -158.1  -21.7
   80.9 -165.2]
[  23.6   65.9 -157.5  -18.1   68.4 -157.9   23.6   75.2 -158.1  -21.7
   80.9 -165.2]
[  23.6   65.9 -157.5  -18.1   68.4 -157.9   23.6   75.2 -158.1  -21.7
   80.9 -165.2]
[  23.6   65.9 -157.5  -18.1   68.4 -157.9   23.6   75.2 -158.1  -21.7
   80.9 -165.2]
[  23.6   65.9 -157.5  -18.1   68.4 -157.9 

KeyboardInterrupt: 